** Problem 1 **: *Censored data fitting* (2 points). 

In [27]:
# data for censored fitting problem
import numpy as np
from numpy.random import randn

n = 2;  # dimension of x's
M = 25;  # number of non-censored data points
K = 100; # total number of points

c_true = randn(n)
X = randn(n, K)
y = np.dot(X.T, c_true) + 0.1 * np.sqrt(n) * randn(K)

# Reorder measurements, then censor
sort_ind = np.argsort(y)
X = X[:, sort_ind];
y = y[sort_ind[:M + 1]]
D = (y[M - 1]+y[M]) / 2
y = y[:M]

The difference between this problem and a classical least squares is that for some of the equations $c_1x_1^i + c_2x_2^i$  we now will have the lower bound $D$. We will introduce $K-M$ values $\Delta y_i$ and will try to fit the model $c_1x_1^i + c_2x_2^i = y_i + \Delta y_i, i > M, \Delta y_i >= 0$

In [34]:
# Solution for part a
from cvxpy import *



** Problem 2 **: *Optimal vehicle speed scheduling (3 points) *-- (additional exercise A3.20 to Boyd and Vandenberghe). 

A vehicle (say, an airplane) travels along a fixed path of *n* segments, between *n + 1* waypoints labeled *0, . . . , n*. Segment *i* starts at waypoint *i − 1* and terminates at waypoint *i*. The vehicle starts at time *t = 0* at waypoint *0*. It travels over each segment at a constant (nonnegative) speed; *si* is the speed on segment *i*. We have lower and upper limits on the speeds: *smin ≤ s ≤ smax*. The vehicle does not stop at the waypoints; it simply proceeds to the next segment. The travel distance of segment *i* is *di* (which is positive), so the travel time over segment *i* is *di/si*. We let *τi*, *i = 1, . . . , n,* denote the time at which the vehicle arrives at waypoint *i*. The vehicle is required to arrive at waypoint *i*, *for i = 1, . . . , n*, between times *τmin,i* and *τmax,i* , which are given. The vehicle consumes fuel over segment *i* at a rate that depends on its speed *Φ(s_i )=a s_i^2+b s_i+c kg/s*.

You are given the data *d* (segment travel distances), *smin* and *smax* (speed bounds), *τmin* and *τmax* (waypoint arrival time bounds), and the the parameters *a*, *b*, and *c* (all parameters are in *veh_speed_sched_data.m*). For the given form of the potentials, find the way to reduce the problem to a convex optimization problem and solve it using CVX (NB: you need not necessarily use one of the “canonical” convex optimization formulations we saw in the course). Use MATLAB command stairs to plot speed vs time for the optimal schedule.  What are relative pros and cons for using convex optimization vs. dynamic programming for such task?

In [33]:
import scipy.io as sio

mat_contents = sio.loadmat('veh_sched_data.mat')

a = mat_contents['a']
b = mat_contents['b']
c = mat_contents['c']
d = mat_contents['d']
n = mat_contents['n']
smin = mat_contents['smin']
smax = mat_contents['smax']
tau_min = mat_contents['tau_min']
tau_max = mat_contents['tau_max']

<img src="3.png" width="65%"/>

__Solution__

Here is Lagrangian:
$$ L(x, \lambda, \nu) = p^Tx + \lambda^T(Ax - b) + \nu^T(d - Cx)$$

$$ g(\lambda, \nu) = \min_x{L(x, \lambda, \nu)} = min_x{\left((p^T + \lambda^TA + \nu^TC)x - \lambda^Tb + \nu^Td\right)}$$

And here is dual problem formulation:

$$\max_{\nu, \lambda}{-\lambda^Tb + \nu^Td}$$
$$s.t.: p^T + \lambda^TA + \nu^TC = 0$$
$$\lambda \geq 0$$

<img src="4.png" width="65%"/>

__Solution__

$$L(x, \lambda, \nu) = f_0(x) + \lambda_1(-x_1 - 2x_2 + 1) + \lambda_2(-3x_1 - x_2 + 1) = x_1^2 + x_2^2 - x_1x_2+ \lambda_1(-x_1 - 2x_2 + 1) + \lambda_2(-3x_1 - x_2 + 1) $$

$$g(\lambda) = \min_{x}{L(x, \lambda)}$$

$$\lambda_i \geq 0, x = (x_1, x_2)^T$$

Dual problem:

$$\max_{\lambda \geq 0}g(\lambda)$$

Let's improve our $L$ representation. First let's observe that $L$ is continuous in every point on all the directions as well as infinetely differentiable. Let's find partial derevatives of $L$ w.r.t $x_1$ and $x_2$:

$$\frac{\partial L}{\partial x_1} = 2x_1-x_2-\lambda_1-3\lambda_2$$
$$\frac{\partial L}{\partial x_2} = 2x_2-x_1-2\lambda_1-\lambda_2$$

since we are minimizing a convex function the following equations should satisfy:

$$\left\{ \begin{array}{rl}
 2x_1-x_2-\lambda_1-3\lambda_2 = 0 \\
 2x_2-x_1-2\lambda_1-\lambda_2 = 0
       \end{array} \right.$$
       
lets now substitute $x_{1,2}$ in our Lagrangian:

<img src="wolfram.png" />

$$L(x, \lambda, \nu) = -\frac{1}{3}\left(7t_1^2+17t_2t_1+13t_2^2\right) + t_1 + t_2, $$
where $t_i = \lambda_i$. 

$$L(x,\lambda,\nu) = -\frac{1}{3}t^T\left( \begin{array}{ccc}
7 & 8.5 \\
8.5 & 13  \end{array} \right)t + t_1 + t_2$$

In [56]:
import cvxpy as cx
from cvxpy import *
import numpy as np

# Initial problem
x = Variable(2, 1)
Q = np.array([1, -0.5, -0.5, 1]).reshape(2, 2)
A = np.array([1, 2, 3, 1]).reshape(2, 2)

obj = Minimize(quad_form(x, Q))
constraints = [ A*x >= 1 ]
prob = Problem(obj, constraints)
res = prob.solve(solver=SCS)
print res

# Dual problem
l = Variable(2, 1)
Q2 = np.array([7, 8.5, 8.5, 13]).reshape(2, 2)
dual_obj = Maximize(-1/3.*quad_form(l, Q2) + l[0] + l[1])
constr = [l >= 0]
prob = Problem(dual_obj, constr)
res = prob.solve(solver=SCS)
print res

0.107141954258
0.107140193577


<img src="5_a.png" width="65%"/>

<img src="5_b.png" width="65%"/>